In [102]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import numpy as np

## 1. Find the 10 countries with most projects
I will extract the countryshortname and corresponding mjthemecode information and create a dataframe consisting of the countryshortname information and transformed mjthemecode information.  Because the mjthemecode column will consist of strings containing the codes, I will apply the str.split() method to transform these entries into lists whose length can be calculated, yielding the number of projects for each country.

NOTE: The greatest number of projects any country has is 5, and there are more than 10 countries with 5 projects.

In [103]:
# Use the pandas.read_json() function to parse the json file directly into a dataframe.
data = pd.read_json('world_bank_projects.json')

In [104]:
# Use the str.split() method to transform the mjthemecode column into a series containing lists of the codes.
#Then apply the len() function replace each list with its length.
project_counts = data.mjthemecode.str.split(',').apply(len)

In [105]:
# Create a new dataframe from the data.countryshortname and project_counts series.
df = pd.DataFrame({'country':data.countryshortname, 'project_counts':project_counts})

In [106]:
# Sort the dataframe by the project_counts column in descending order.
# Slice the first 10 entries of the country column to obtain the top 10 countries.
df.sort_values('project_counts', ascending=False).country[0:10]

219                    Niger
159    Sao Tome and Principe
75           Kyrgyz Republic
388               Tajikistan
390                   Jordan
391                  Tunisia
397                Indonesia
155                 Mongolia
64               Afghanistan
290                   Rwanda
Name: country, dtype: object

## 2. Find the top 10 major project themes: 
### Method 1:
I will use the json library to load the json file.  Then I will use json_normalize() to extract just the mjtheme_namecode information, since I'm only interested in obtaining counts of the occurence of each code.

In this first method I will simply use the value_counts method on the resulting dataframe to obtain the occurences of each code in descending order.

In [107]:
# Use the context manager to open and close a connection to the file.
# Load the json to the variable, data.
with open('world_bank_projects.json', 'r') as file:
    data = json.load(file)

In [108]:
# Use the json_normalize() function to extract only the mjtheme_namecode information.
mj_theme_df = json_normalize(data, 'mjtheme_namecode')

In [109]:
# We are interested in the occurence of the codes in the resulting code column.
# Use the value_counts() method on the code column and slice the first 10 results to obtain the top 10.
mj_theme_df.code.value_counts()[0:10]

11    250
10    216
8     210
2     199
6     168
4     146
7     130
5      77
9      50
1      38
Name: code, dtype: int64

## 2. Find the top 10 major project themes: 
### Method 2:
In this second method I will chain the groupby() and sort_values() methods to obtain the occurences of each code in stead of the value_counts() method.

In [110]:
# In stead of using the value_counts() method group by the values of the code column and us the
# aggregate function count() to obtain totals corresponding to each code.
# Chain the sort_values() method in descending order and slice the first 10 results to obtain the top 10.
mj_theme_df.groupby('code').count().sort_values('name', ascending=False)[0:10]

,name
code,
11,250
10,216
8,210
2,199
6,168
4,146
7,130
5,77
9,50


## 3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

### Method 1: 
I will create a dictionary mapping each code to its corresponding name.  Then I will define a function that takes a list of dictionaries and returns a list of tuples, using the dictionary I will have created to replace the '' entries with the appropriate names.

In [111]:
# Use the pandas.read_json() function to read the json file into a dataframe.
data = pd.read_json('world_bank_projects.json')

In [112]:
# For convenience, replace the '' entries in the name column of the mj_theme_df dataframe used previously.
mj_theme_df.name = mj_theme_df.name.replace('', np.nan)

In [113]:
# Obtain a series containing unique entries for each code with coresponding name by filtering out the null values
# and then dropping duplicate entries.
# Then create a dictionary mapping the codes to the names.
dup_dropped = mj_theme_df.loc[mj_theme_df.name.notnull(), :].drop_duplicates()
namecode_dict = dict(zip(dup_dropped.code, dup_dropped.name))

In [114]:
# Define a function to replace each list of dictionaries with lists of tuples containing the code
# and corresponding name by utilizing the namecode_dict created previously.
def replace_name(x):
    ret_list = []
    for each in x:
        code = each['code']
        name = namecode_dict[code]
        ret_list.append((int(code),name))
    return ret_list

In [115]:
# Apply the previously defined function along the mjtheme_namecode column.
data.mjtheme_namecode = data.mjtheme_namecode.apply(replace_name)

In [116]:
# Observe the desired results.
data.mjtheme_namecode

0      [(8, Human development), (11, Environment and ...
1      [(1, Economic management), (6, Social protecti...
2      [(5, Trade and integration), (2, Public sector...
3      [(7, Social dev/gender/inclusion), (7, Social ...
4      [(5, Trade and integration), (4, Financial and...
5      [(6, Social protection and risk management), (...
6      [(2, Public sector governance), (4, Financial ...
7      [(11, Environment and natural resources manage...
8      [(10, Rural development), (7, Social dev/gende...
9      [(2, Public sector governance), (2, Public sec...
10     [(10, Rural development), (2, Public sector go...
11     [(10, Rural development), (6, Social protectio...
12       [(4, Financial and private sector development)]
13     [(5, Trade and integration), (11, Environment ...
14     [(6, Social protection and risk management), (...
15     [(10, Rural development), (11, Environment and...
16     [(10, Rural development), (2, Public sector go...
17     [(8, Human development),

## 3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.
### Method 2:
I will use json_normalize to parse the countryshortname and mjtheme_namecode information.  From the resulting dataframe I will filter out the rows containing '' in the name column and remove duplicate entries thereby creating a new dataframe containing unique rows with code and name information.  I will then remove the name column from the original dataframe and merge the two dataframes on the code column creating the desired result.

In [117]:
# Using the context manager to open and close the file connection, load the contents of the json file
# into the variable, data.
with open('world_bank_projects.json', 'r') as file:
    data = json.load(file)

In [118]:
# Use json_normalize to extract the mjtheme_namecode and countryshortname information.
df = json_normalize(data, 'mjtheme_namecode', ['countryshortname'])

In [119]:
# Create a new datafame where the rows containing '' in the name column are filtered out and duplicates
# are dropped.
to_merge = df[['code', 'name']].loc[df.name != ''].drop_duplicates()

In [120]:
# Switch the code and countryshortname columns because I prefer it that way.  Exclude the name column.
df = df[['countryshortname', 'code']]

In [121]:
# Left outer join the two dataframes on the code column to create a new name column containing the
# appropriate names.
df_merged = pd.merge(on='code', left=df, right=to_merge, how='left')

In [122]:
# Observe the desired results.
df_merged

,countryshortname,code,name
0,Ethiopia,8,Human development
1,Ethiopia,11,Environment and natural resources management
2,Tunisia,1,Economic management
3,Tunisia,6,Social protection and risk management
4,Tuvalu,5,Trade and integration
5,Tuvalu,2,Public sector governance
6,Tuvalu,11,Environment and natural resources management
7,Tuvalu,6,Social protection and risk management
8,"Yemen, Republic of",7,Social dev/gender/inclusion
9,"Yemen, Republic of",7,Social dev/gender/inclusion
